In [ ]:
# import packages
import pandas as pd
import ixmp
import message_ix

from utils.run_scenarios import *
from utils.pp_db_to_xlsx import results_to_xlsx
from utils.pp_plot_emissions import plot_emissions
from utils.pp_plot_power_sector import plot_power_sector
from utils.pp_plot_heatmaps import plot_heatmap_comparison
#from utils.getResultsFromScenario.py import *

def getResultsFromScenario(Model):
	emissions = Model.var('EMISS')
	act = Model.var('ACT')
	cap = Model.var('CAP')
	
	ScenarioSpecificResults = ScenarioResults(emissions, act, cap)
	return ScenarioSpecificResults

class ScenarioResults:
	def __init__(self, emissions, act, cap):
		self.emissions = emissions
		self.capacity = cap
		self.activity = act
	

In [ ]:
# define database and the baseline scenario
model = 'MESSAGE South Africa'
baseline = 'baseline'
database = 'message_sa'

In [ ]:
# launch a platform to access the database
mp = ixmp.Platform(dbprops=f'db/{database}', dbtype='HSQLDB')
base = message_ix.Scenario(mp, model=model, scenario=baseline)
#base.solve(model='MESSAGE')

In [ ]:
# run baseline scenario (if not done yet)
base.solve(model='MESSAGE')

In [ ]:
# clone baseline scenario and check out for modification
scenario = "baseline_new"
scen = base.clone(model, scenario, keep_solution=False)
scen.set_as_default()
scen.check_out()

In [ ]:
# check which parameters exist
scen.par_list()

In [ ]:
# read demand parameter into dataframe
df = scen.par('demand')

In [ ]:
# write demand dataframe to csv file 
file_name = 'demand_out.csv'
df.to_csv(file_name)

In [ ]:
# read demand dataframe from csv
filename_in = "demand_in.csv"
df_new = pd.read_csv(filename_in)

In [ ]:
print(df_new)
print(df)

In [ ]:
# add new demand to ixmp 
scen.add_par('demand', df_new)

In [ ]:
# commit changed scenario to ixmp
scen.commit('adjusted South Africa demand')

In [ ]:
# solve changed South Africa scenario
scen.solve()

In [ ]:
# look at activity variable
print(scen.var('ACT'))
print(base.var('ACT'))

In [ ]:
#print(scen.par('demand'))
#print(base.par('demand'))
#scen.to_excel('testOut.xlsx')
#shale_cost = [1, 10, 30, 1000]
#carbon_cost = [0, 5, 15, 30]
#results_to_xlsx(base, baseline, database, shale_cost, carbon_cost)
#scen.var_list()


test_ssp5 = getResultsFromScenario(scen)
test_ssp2 = getResultsFromScenario(base)
print(test_ssp2.emissions)
print(test_ssp5.emissions)


In [ ]:
print( scen.var('EMISS'))

In [ ]:
# close ixmp connection
mp.close_db()